# 32.07 Búsqueda bayesiana - Walk Forward Validation - XGBoost
### Librerias usadas por el programa

In [5]:
#limpio la memoria
rm(list=ls())
gc()

library("data.table")
library("ROCR")
library("xgboost")
library("caret")
library("DiceKriging")
library("mlrMBO")

library( "Rcpp" )


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2358443,126.0,4294378,229.4,4294378,229.4
Vcells,3963245,30.3,8388608,64.0,6921734,52.9


## Parámetros generales

In [6]:
#Archivo con datos totales (etiquetados hasta 201912 + parciales 202001 + sin etiquetar 202002)
#karchivo_entrada      <-  "~/buckets/b1/datasetsOri/paquete_premium_faseII.txt.gz"
karchivo_entrada <- "predsAnteriores"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

kdir_trabajo <- "~/buckets/b1/work/"

kdir_salida <- "~/buckets/b1/work/"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kclase_nomcampo)

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase01"

#Identificación del modelo
kmodelo               <-  "32.14-Stacking-WFV-XGB"


#Parametros  mlrMBO
mbo <- list()
mbo$iteraciones       <-  300
mbo$saveondisk_time   <-  600   # cada 600 segundos guarda a disco cuanto avanzo
mbo$archivo_trabajo   <-  paste(kdir_trabajo,"exp_", kmodelo, ".RDATA", sep="") 


#estos valores se graban en el archivo de salida
hiper <- list()
hiper$experimento    <-  kmodelo
hiper$arch_global    <-  paste(kdir_trabajo, "hiperparametro_GLOBAL.txt", sep="" )
hiper$arch_local     <-  paste(kdir_trabajo, "hiperparametro_", kmodelo, ".txt", sep="" )
hiper$arch_imagen    <-  paste(kdir_trabajo, "exp_", kmodelo, ".jpg", sep="" )
hiper$arch_over      <-  paste(kdir_trabajo, "exp_", kmodelo, "_over.jpg", sep="" )
hiper$directory      <-  kdir_trabajo
hiper$clase_tipo     <-  "ternaria"
hiper$programa       <-  "32.14-Stacking-WFV-XGB"
hiper$algoritmo      <-  "XGBoost"
hiper$busqueda       <-  "MBO"
hiper$estimacion     <-  ""
hiper$observaciones  <-  "12 Meses"
hiper$separador      <-  "\t"


problema <- list()
problema$prob_corte           <-      0.025
problema$ganancia_TP     <-  29250 
problema$ganancia_FP   <-   -750

offset <- 0

## Funciones auxiliares y preparación de optimización bayesiana

### Escritura de archivos de progreso

In [7]:
#------------------------------------------------------

hiperparametros_titulos = function( phiper, parchivo )
{

    cat("experimento",
        "metrica1_actual",
        "metrica2_actual",
        "tiempo",
        "pcorte",
        "parametros",
        "fecha", 
        "clase", "programa", "algoritmo", "busqueda" , "estimacion",
        "dataset_actual_train", "observaciones",
        "\n", 
        sep=phiper$separador, 
        file=parchivo, 
        fill=FALSE, 
        append=FALSE)

}
#------------------------------------------------------
#hiperparametros_crear = function( phiper)
#{
#  hiperparametros_titulos( phiper, phiper$arch_global ) 
#  hiperparametros_titulos( phiper, phiper$arch_local ) 
#}
#------------------------------------------------------
hiperparametros_grabar1 = function( phiper, parchivo,
                                    pactual_ganancia, pactual_auc, 
                                    ptiempo, pprob_corte, pst_parametros, 
                                    parchivo_actual_train,  pclase)
{

  if(!file.exists(parchivo))
  {   hiperparametros_titulos( phiper, parchivo ) }

  cat( phiper$experimento,
       pactual_ganancia, 
       pactual_auc,
       ptiempo,
       pprob_corte,
       pst_parametros,
       format(Sys.time(), "%Y%m%d %H%M%S"), 
       ifelse( is.na( pclase), phiper$clase_tipo,  pclase ),
       phiper$programa, 
       phiper$algoritmo, 
       phiper$busqueda, 
       phiper$estimacion,
       karchivo_entrada, 
       phiper$observaciones,
       "\n", 
      sep=phiper$separador, 
      file=parchivo, 
      fill=FALSE, append=TRUE 
    )

}
#------------------------------------------------------------------------------

hiperparametros_grabar = function( phiper,
                                   pactual_ganancia, pactual_auc,
                                   ptiempo, pprob_corte, pst_parametros, 
                                   parchivo_actual_train,  pclase)
{
  #grabo el archivo  GLOBAL
  hiperparametros_grabar1(phiper, phiper$arch_global,
                          pactual_ganancia, pactual_auc, 
                          ptiempo, pprob_corte, pst_parametros, 
                          parchivo_actual_train,  pclase )

  #grago el archivo  LOCAL
  hiperparametros_grabar1(phiper, phiper$arch_local,
                          pactual_ganancia, pactual_auc, 
                          ptiempo, pprob_corte, pst_parametros, 
                          parchivo_actual_train,  pclase )

}

### Cálculo de ganancia

In [8]:
fganancia_logistic_xgboost   <- function(probs, clases) 
{

  vlabels <- getinfo(clases, "label")

  gan <- as.numeric(sum((probs > 0.025  ) * 
                ifelse( vlabels== 1, +29250, -750 )   
           ))/length(vlabels) * 200000

  return(  list(metric = "ganancia", value =  ifelse(  is.na(gan) , 0, gan ) )  )
}

### Cargo Archivos

In [9]:
train_201811_201910 <- fread(paste0("~/buckets/b1/work/Predictions/final_", 201811, "_", 201910, "_preds.csv.gz"))
train_201810_201909 <- fread(paste0("~/buckets/b1/work/Predictions/final_", 201810, "_", 201909, "_preds.csv.gz"))
train_201807_201906 <- fread(paste0("~/buckets/b1/work/Predictions/final_", 201807, "_", 201906, "_preds.csv.gz"))

test_201912 <- fread(paste0("~/buckets/b1/work/Predictions/final_", 201912, "_preds.csv.gz"))
test_201911 <- fread(paste0("~/buckets/b1/work/Predictions/final_", 201911, "_preds.csv.gz"))
test_201908 <- fread(paste0("~/buckets/b1/work/Predictions/final_", 201908, "_preds.csv.gz"))

### Prueba Walk Forward Validation

In [10]:
mesesWFV <- data.frame(mes_desde=c(201811,201810,201807), mes_hasta=c(201910,201909,201906), mes_test=c(201912,201911,201908))

for ( i in 1:3){

mes_test  <- mesesWFV[i, 'mes_test']
mes_hasta <- mesesWFV[i, 'mes_hasta']
mes_desde <- mesesWFV[i, 'mes_desde']
print(paste("Entreno de",mes_desde,"a",mes_hasta,"y testeo en",mes_test))
    
}

[1] "Entreno de 201811 a 201910 y testeo en 201912"
[1] "Entreno de 201810 a 201909 y testeo en 201911"
[1] "Entreno de 201807 a 201906 y testeo en 201908"


### Función de ganancia a invocar en búsqueda bayesiana

In [11]:
modelo_xgboost_ganancia = function(x = list(
                                            subsample,
                                            colsample_bytree,
                                            eta,
                                            min_child_weight,
                                            gamma,
                                            max_depth,
                                            alpha,
                                            lambda,
                                            max_leaves
                                            ) )
    {
    t0   <-  Sys.time()

    #Inicializo totales de gnancia, auc e iteraciones
    gan_add = 0
    auc_add = 0
    best_iter_add = 0

    #determino total de meses
    mesesWFV <- data.frame(mes_desde=c(201811,201810,201807), mes_hasta=c(201910,201909,201906), mes_test=c(201912,201911,201908))

    for ( i in 1:3){
        mes_test  <- mesesWFV[i, 'mes_test']
        mes_hasta <- mesesWFV[i, 'mes_hasta']
        mes_desde <- mesesWFV[i, 'mes_desde']

        print(paste("Entreno de",mes_desde,"a",mes_hasta,"y testeo en",mes_test))        

        #train <- fread(paste0("~/buckets/b1/work/Predictions/final_", mes_desde, "_", mes_hasta, "_preds.csv.gz"))
        train <- get(eval(paste0("train_", mes_desde, "_",mes_hasta)))
        y.train <- as.matrix(train[,..kobjetivo])
        X.train <- as.matrix(train[,-..kobjetivo])

        #test <- fread(paste0("~/buckets/b1/work/Predictions/final_", mes_test, "_preds.csv.gz"))
        test <- get(eval(paste0("test_", mes_test)))
        y.test <- as.matrix(test[,..kobjetivo])
        X.test <- as.matrix(test[,-..kobjetivo])

        dtrain <- xgb.DMatrix(X.train, label = y.train) 
        dtest <- xgb.DMatrix(X.test , label = y.test )  

        modelo <- xgb.train(data= dtrain,
                        objective= "binary:logistic",
                        feval= fganancia_logistic_xgboost,
                        maximize= TRUE,
                        watchlist= list( test=dtest ),                   
                        base_score=mean(getinfo(dtrain, "label")), #Prob de corte del PROBLEMA
                        nrounds= 999999999,  #un numero muy grande
                        early_stopping_rounds= 100,
                        tree_method= "hist",
                        max_bin= 1023,
                        subsample=x$subsample,
                        colsample_bytree=x$colsample_bytree,
                        eta=x$eta,
                        min_child_weight=x$min_child_weight,
                        gamma=x$gamma,
                        max_depth=x$max_depth,
                        alpha=x$alpha,
                        lambda=x$lambda,
                        max_leaves=x$max_leaves                          
                       )


        t1   <-  Sys.time()
        tiempo_corrida <-  as.numeric( t1 - t0, units = "secs")

        #Obtengo ganancia paso
        gan <- modelo$best_score
        best_iter <- modelo$best_iteration
        auc <- 1

        #Acumulo en totales de WFV
        gan_add = gan/length(mesesWFV) + gan_add
        auc_add = auc/length(mesesWFV) + auc_add
        best_iter_add = best_iter/length(mesesWFV) + best_iter_add

    }

    #genero el string con los parametros
    st_parametros = paste(
                        "subsample=",x$subsample, ", " ,
                        "colsample_bytree=",x$colsample_bytree, ", " ,
                        "eta=",x$eta, ", " ,
                        "min_child_weight=",x$min_child_weight, ", " ,
                        "gamma=",x$gamma, ", " ,
                        "max_depth=",x$max_depth, ", " ,
                        "alpha=",x$alpha, ", " ,
                        "lambda=",x$lambda, ", " ,
                        "max_leaves=",x$max_leaves, ", " ,
                        "iters=",        best_iter_add,
                        sep = ""
                       )

    #escribo al archivo de salida los resultados de esta corrida
    hiperparametros_grabar( phiper= hiper,
                          pactual_ganancia= gan_add, 
                          pactual_auc= auc_add,
                          ptiempo= tiempo_corrida,
                          pprob_corte= problema$prob_corte, 
                          pst_parametros= st_parametros, 
                          parchivo_actual_train= karchivo_entrada,
                          pclase= 'ternaria'
                        )

    return(-gan_add)
  
}

In [12]:
configureMlr(show.learner.output = FALSE)
funcion_optimizar <-  modelo_xgboost_ganancia

### Espacio de búsqueda bayesiana

In [13]:
#configuro la busqueda bayesiana, los hiperparametros que se van a optimizar
obj.fun <- makeSingleObjectiveFunction(
        name = "prueba",
        fn   = funcion_optimizar,
        par.set = makeParamSet(
            makeNumericParam("subsample"         ,  lower=0.01    , upper=1), #bagging_fraction lightGBM
            makeNumericParam("colsample_bytree"         ,  lower=0.01    , upper=1), #feature_fraction
            makeNumericParam("eta"         ,  lower=.01    , upper=.5),#learning_rate
            makeNumericParam("min_child_weight"         ,  lower=0    , upper=0.02), 
            #makeIntegerParam("min_data_in_leaf"         ,  lower=1L    , upper=1000L), #no existe en XGBoost
            makeNumericParam("gamma"         ,  lower=0    , upper=1),  #min_gain_to_split         
            makeIntegerParam("max_depth"         ,  lower=1L    , upper=10L),
            makeNumericParam("alpha"         ,  lower=0    , upper=5), #lambda_l1
            makeNumericParam("lambda"         ,  lower=0    , upper=100), #lambda_l2
            #makeIntegerParam("max_bin"         ,  lower=1022L    , upper=1023L),
            makeIntegerParam("max_leaves"         ,  lower=4L    , upper=4096L) #num_leaves
        ),
        has.simple.signature = FALSE,
        global.opt.value = -1
        )

In [14]:
#Modificado 1/11 - Se deja de usar el RDATA de avance y en caso de continuar usa el log de búsqueda .csv como "hint"
ctrl <-  makeMBOControl(propose.points = 1L)
ctrl <-  setMBOControlTermination(ctrl, iters = mbo$iteraciones )
ctrl <-  setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI(), opt  = "focussearch", opt.focussearch.points = mbo$iteraciones)
surr.km <-  makeLearner("regr.km", predict.type = "se", covtype = "matern3_2", control = list(trace = FALSE))


#ctrl <-  makeMBOControl(propose.points = 1L, save.on.disk.at.time = mbo$saveondisk_time,  save.file.path = mbo$archivo_trabajo)
#ctrl <-  setMBOControlTermination(ctrl, iters = mbo$iteraciones )
#ctrl <-  setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI(), opt  = "focussearch", opt.focussearch.points = mbo$iteraciones)
#surr.km <-  makeLearner("regr.km", predict.type = "se", covtype = "matern3_2", control = list(trace = FALSE))

### Inicio/continúo búsqueda bayesiana

In [ ]:
#Modificado 1/11 - Se deja de usar el RDATA de avance y en caso de continuar usa el log de búsqueda .csv como "hint"
if(!file.exists(hiper$arch_local))
{
    #lanzo la busqueda bayesiana
    run  <-  mbo(obj.fun, learner = surr.km, control = ctrl)
} else {
    #Lee archivo de avance
    tbl <- fread(hiper$arch_local)

    #Genero hint para MBO con corridas pasadas
    prev_runs <- data.frame()
    for (run in 1:nrow(tbl)) {
        run_p <- list()
        run_p$y <- -tbl[run,metrica1_actual] #cambio el signo porque esta configurado para minimizar la ganancia
        eval(parse(text=paste0('run_p$',trimws(unlist(strsplit(tbl[run,parametros],','))))))
        prev_runs <- rbind(prev_runs,run_p)
    }

    #Descarto iters porque no es parte del MBO
    prev_runs$iters<-NULL
    
    #retoma el procesamiento en donde lo dejo
    run  <-  mbo(obj.fun, learner = surr.km, control = ctrl, design=prev_runs)
}


#if(!file.exists(mbo$archivo_trabajo))
#{
#  #lanzo la busqueda bayesiana
#  run  <-  mbo(obj.fun, learner = surr.km, control = ctrl)
#} else {
#
#  #retoma el procesamiento en donde lo dejo
#  run <- mboContinue( mbo$archivo_trabajo )
#}




Computing y column(s) for design. Not provided.



[1] "Entreno de 201811 a 201910 y testeo en 201912"
[1]	test-ganancia:13825975.902001 
Will train until test_ganancia hasn't improved in 100 rounds.

[2]	test-ganancia:457039.265676 
[3]	test-ganancia:457039.265676 
[4]	test-ganancia:457039.265676 
[5]	test-ganancia:457039.265676 
[6]	test-ganancia:457039.265676 
[7]	test-ganancia:137757.518069 
[8]	test-ganancia:137757.518069 
[9]	test-ganancia:83945.987573 
[10]	test-ganancia:-4304.922440 
[11]	test-ganancia:0.000000 
[12]	test-ganancia:11479.793172 
[13]	test-ganancia:856679.565490 
[14]	test-ganancia:1113539.937722 
[15]	test-ganancia:1605736.069988 
[16]	test-ganancia:1605736.069988 
[17]	test-ganancia:1825287.114410 
[18]	test-ganancia:1551207.052420 
[19]	test-ganancia:1741341.126837 
[20]	test-ganancia:1803762.502212 
[21]	test-ganancia:1942237.507354 
[22]	test-ganancia:2053448.003712 
[23]	test-ganancia:2108694.508354 
[24]	test-ganancia:2191923.008854 
[25]	test-ganancia:2142416.400798 
[26]	test-ganancia:2309590.888871 
[27

[mbo] 36: subsample=0.643; colsample_bytree=0.0543; eta=0.407; min_child_weight=0.00386; gamma=0.197; max_depth=2; alpha=2.8; lambda=76.7; max_leaves=1615 : y = -1.19e+07 : 70.5 secs : infill_ei



[1] "Entreno de 201811 a 201910 y testeo en 201912"
[1]	test-ganancia:13483017.080976 
Will train until test_ganancia hasn't improved in 100 rounds.

[2]	test-ganancia:2774522.512353 
[3]	test-ganancia:2774522.512353 
[4]	test-ganancia:2774522.512353 
[5]	test-ganancia:2774522.512353 
[6]	test-ganancia:2643939.865017 
[7]	test-ganancia:2625285.201112 
[8]	test-ganancia:2542056.700612 
[9]	test-ganancia:2520532.088414 
[10]	test-ganancia:2521249.575487 
[11]	test-ganancia:2521249.575487 
[12]	test-ganancia:2521249.575487 
[13]	test-ganancia:2521249.575487 
[14]	test-ganancia:2546361.623051 
[15]	test-ganancia:2546361.623051 
[16]	test-ganancia:2546361.623051 
[17]	test-ganancia:2551384.032564 
[18]	test-ganancia:2524119.523780 
[19]	test-ganancia:2524119.523780 
[20]	test-ganancia:2524837.010853 
[21]	test-ganancia:2524837.010853 
[22]	test-ganancia:2524837.010853 
[23]	test-ganancia:2524837.010853 
[24]	test-ganancia:2525554.497926 
[25]	test-ganancia:2524837.010853 
[26]	test-ganancia

[mbo] 37: subsample=0.313; colsample_bytree=0.17; eta=0.439; min_child_weight=0.00669; gamma=0.29; max_depth=2; alpha=1.2; lambda=49.2; max_leaves=1466 : y = -1.18e+07 : 65.5 secs : infill_ei



...

[mbo] 229: subsample=0.402; colsample_bytree=0.547; eta=0.488; min_child_weight=0.0126; gamma=0.116; max_depth=3; alpha=1.09; lambda=37.6; max_leaves=2095 : y = -1.11e+07 : 71.1 secs : infill_ei



[1] "Entreno de 201811 a 201910 y testeo en 201912"
[1]	test-ganancia:14446602.220383 
Will train until test_ganancia hasn't improved in 100 rounds.

[2]	test-ganancia:35874.353664 
[3]	test-ganancia:0.000000 
[4]	test-ganancia:0.000000 
[5]	test-ganancia:0.000000 
[6]	test-ganancia:0.000000 
[7]	test-ganancia:0.000000 
[8]	test-ganancia:0.000000 
[9]	test-ganancia:0.000000 
[10]	test-ganancia:0.000000 
[11]	test-ganancia:0.000000 
[12]	test-ganancia:0.000000 
[13]	test-ganancia:0.000000 
[14]	test-ganancia:0.000000 
[15]	test-ganancia:0.000000 
[16]	test-ganancia:0.000000 
[17]	test-ganancia:0.000000 
[18]	test-ganancia:0.000000 
[19]	test-ganancia:0.000000 
[20]	test-ganancia:0.000000 
[21]	test-ganancia:0.000000 
[22]	test-ganancia:0.000000 
[23]	test-ganancia:0.000000 
[24]	test-ganancia:0.000000 
[25]	test-ganancia:0.000000 
[26]	test-ganancia:0.000000 
[27]	test-ganancia:0.000000 
[28]	test-ganancia:0.000000 
[29]	test-ganancia:0.000000 
[30]	test-ganancia:0.000000 
[31]	test-gan

[mbo] 230: subsample=0.347; colsample_bytree=0.421; eta=0.39; min_child_weight=0.00248; gamma=0.765; max_depth=1; alpha=3.07; lambda=53.3; max_leaves=357 : y = -1.31e+07 : 65.9 secs : infill_ei



[1] "Entreno de 201811 a 201910 y testeo en 201912"
[1]	test-ganancia:13161582.872149 
Will train until test_ganancia hasn't improved in 100 rounds.

[2]	test-ganancia:-157847.156120 
[3]	test-ganancia:-157847.156120 
[4]	test-ganancia:-157847.156120 
[5]	test-ganancia:-157847.156120 
[6]	test-ganancia:-157847.156120 
[7]	test-ganancia:-157847.156120 
[8]	test-ganancia:-32286.918297 
[9]	test-ganancia:-32286.918297 
[10]	test-ganancia:-32286.918297 
[11]	test-ganancia:-32286.918297 
[12]	test-ganancia:-24394.560491 
[13]	test-ganancia:-25112.047565 
[14]	test-ganancia:-21524.612198 
[15]	test-ganancia:-14349.741465 
[16]	test-ganancia:-12914.767319 
[17]	test-ganancia:-12914.767319 
[18]	test-ganancia:-12914.767319 
[19]	test-ganancia:-12914.767319 
[20]	test-ganancia:-12914.767319 
[21]	test-ganancia:-12914.767319 
[22]	test-ganancia:-12914.767319 
[23]	test-ganancia:-12914.767319 
[24]	test-ganancia:-12914.767319 
[25]	test-ganancia:-12197.280246 
[26]	test-ganancia:-12197.280246 
[2

Warning message in close.connection(file):
“Problem closing connection:  Input/output error”
[mbo] 231: subsample=0.516; colsample_bytree=0.152; eta=0.485; min_child_weight=0.0151; gamma=0.789; max_depth=1; alpha=2.28; lambda=78.4; max_leaves=3313 : y = -1.25e+07 : 69.5 secs : infill_ei



In [ ]:
ls()

## Genero predicción

### Identifico mejor solución

In [ ]:
tbl <- fread(hiper$arch_local)
mejor_hiper <- tbl[1,parametros]
setorder( tbl, -metrica1_actual)
mejor_hiper <- tbl[1,parametros]

best_p = list()
eval(parse(text=paste0('best_p$',trimws(unlist(strsplit(mejor_hiper,','))))))

In [ ]:
best_p

### Genero dataset de entrenamiento

In [ ]:
t0   <-  Sys.time()
total_meses <- length(meses)

mes_predict <- meses[total_meses]
mes_hasta <- meses[(total_meses-2)]
mes_desde <- meses[(total_meses-1-best_p$q_meses)]

In [ ]:
# Determina decay desde el último mes de la lista (ajustar el 0.95 como hiperparametro)
mapa_pesos = data.table(foto_mes = meses, weight= best_p$decay^order(-meses))

#Determina una lista de pesos coincidente con el dataset
pesos = merge(dataset[,c("foto_mes")],mapa_pesos,on=foto_mes,sort=FALSE)[,weight]

#Filtro pesos por meses a usar en entrenamiento
weights <- pesos[dataset$foto_mes >= mes_desde & dataset$foto_mes <= mes_hasta]

#Filtro dataset de entrenamiento
train <- dataset[foto_mes >= mes_desde & foto_mes <= mes_hasta]
y.train <- as.matrix(train[,..kobjetivo])
X.train <- as.matrix(train[,-..kobjetivo])

#genero el formato requerido por LightGBM
dtrain  <-   lgb.Dataset( data  = X.train,
                               label = y.train, 
                               missing=NA,
                               free_raw_data=FALSE,
                               weight=weights
                              )

### Entreno modelo

In [ ]:
modelo = lgb.train( 
               data = dtrain,
               objective = "binary",
               nrounds= round(best_p$iters),
#               eval=fmetrica_ganancia,
#               metric=c("ganancia", "auc"),
#               early_stopping_rounds=100,
               boost_from_average=TRUE,
               bagging_fraction=best_p$bagging_fraction, 
               feature_fraction=best_p$feature_fraction, 
               learning_rate=best_p$learning_rate,  
               min_child_weight=best_p$min_child_weight, 
               min_data_in_leaf=best_p$min_data_in_leaf, 
               min_gain_to_split=best_p$min_gain_to_split,     
               max_depth=best_p$max_depth, 
               lambda_l1=best_p$lambda_l1, 
               lambda_l2=best_p$lambda_l2, 
               max_bin=best_p$max_bin, 
               num_leaves=best_p$num_leaves 
              )

t1   <-  Sys.time()
tiempo_corrida <-  as.numeric( t1 - t0, units = "secs")

### Genero predicción y grabo

In [ ]:
#cargo los datos
dpredict <- dataset[foto_mes == mes_predict]

#Armo dataset de salida
dataset_salida <- dpredict[,..kcampo_id]
setnames(dataset_salida, c("Id") )

In [ ]:
#aplico el modelo a los datos sin clase, 202002
Preds  <- predict(modelo,as.matrix(dpredict[,-..kobjetivo]))

In [ ]:
#Selecciono los que estan por encima del umbral
dpredict$Predicted <- Preds > problema$prob_corte  + best_p$offset

#Grabo salida
dataset_salida[,"Predicted"] <- dpredict$Predicted
write.csv(dataset_salida,paste0(kdir_salida, kmodelo, "_salida_",nrow(tbl),".csv"), row.names = FALSE)

In [ ]:
#Minimo Score
min(Preds)

In [ ]:
#Maximo Sco
max(Preds)